### Retrieval Scoring + ranking


In [1]:
## Init stuff
import os
import json
import psycopg2
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # reads .env if present

REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT") or "knowledge-collection-service/0.1"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- connect to Postgres ---
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="knowledgebase",
    user="stiw_user",
    password="stiw_pwd"
)
cursor = conn.cursor()
print("✅ Connected to Postgres and OpenAI")

✅ Connected to Postgres and OpenAI


Run once for Query & Query Embed, & test other systems for tweaks


In [20]:
user_input = input("Enter your query or goal: ").strip()
print(f"Query: --> {user_input}")

# Running HyDE style query improvement 

HyDE_prompt = f"""
You are an extremely empathetic Reddit user writing a raw, honest submission asking for help.
Write a detailed (150–250 word) Reddit post (title + body) from someone in exactly the same situation as the user below.
Make it the kind of thoughtful, non-ranty, "I know I need to change, please help me" post that gets tons of supportive, practical, life-changing advice in the comments.

Query / situation: {user_input}

Start directly with the post (include a realistic title and then the body). 
Do NOT write advice or a success story — only the help-seeking post.
"""

print("Creating a HyDE hypothetical Reddit submission (OP post)...")
HyDE_response = client.chat.completions.create(
    model="gpt-4o-mini",       
    messages=[
        {"role": "system", "content": "You are a Reddit user asking for help in a detailed, vulnerable way."},
        {"role": "user", "content": HyDE_prompt}
    ],
    temperature=0.8, # keeps it varied and natural
    max_tokens=500 # Wanna limit token usage here to create precise embeddings
)

HyDE_answer = HyDE_response.choices[0].message.content.strip()

print("\📄 HyDE generated hypothetical Reddit submission \n")
print(HyDE_answer)

# Embedd HyDE, query
q_emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=HyDE_answer
    ).data[0].embedding
print("✅ Query Embedding generated...")

Query: --> I wanna be gay and ride superman
Creating a HyDE hypothetical Reddit submission (OP post)...
\📄 HyDE generated hypothetical Reddit submission 

**Title: Struggling with My Identity and Seeking Support**

Hey Reddit, 

I find myself in a confusing and vulnerable place, and I hope sharing my feelings here might help me find some clarity. I’m a 28-year-old who has always identified as straight, but lately, I’ve been questioning my sexuality. I’ve been feeling drawn to the idea of identifying as gay, and it’s both exciting and terrifying for me. I want to explore this part of myself, but I’m struggling with the fear of judgment from those around me, including family and friends. 

On top of that, I've been grappling with a long-held dream: I want to ride a roller coaster called "Superman" at my local amusement park. It sounds silly, but I’ve always been too scared to go on it, partly because I fear how I’ll be perceived by others. I see this ride as a metaphor for the freedom I 

In [21]:
print(q_emb)

[0.048742491751909256, -0.02118801139295101, -0.014142117463052273, 0.06275878846645355, -0.006438815500587225, -0.017249859869480133, -0.0012353906640782952, 0.025251982733607292, 0.005092546343803406, 0.06280911713838577, -0.006781673524528742, -0.013802404515445232, -0.03920537233352661, 0.017614737153053284, 0.02851071022450924, -0.018055105581879616, -0.0037494224961847067, 0.07312632352113724, 0.011657181195914745, 0.05994043126702309, 0.04348323494195938, -0.012147877365350723, 0.0054857321083545685, 0.009203699417412281, -0.008159398101270199, -0.025793006643652916, 0.009172244928777218, -0.00124089524615556, -0.004809452220797539, 0.0076938653364777565, -0.021829690784215927, -0.03417259082198143, -0.04275348037481308, 0.005237238481640816, -0.01867162063717842, 0.0172876063734293, -0.04408716782927513, -0.02614530175924301, 0.08455073833465576, -0.015752607956528664, 0.03978414088487625, -0.022043583914637566, 0.06004108861088753, 0.01363883912563324, -0.020785389468073845, 0

Running through retrieved chunks for new scoring


In [ ]:
print("Retrieveing top matching submissions....")
cursor.execute("""
            SELECT 
                submission_id,
                domain_id,
                title, 
                selftext, 
                permalink, 
                (embedding <=> %s::vector) AS distance
            FROM submissions
            ORDER BY embedding <=> %s::vector
            LIMIT 15;
        """, (json.dumps(q_emb), json.dumps(q_emb)))
top_subs = cursor.fetchall()
top_subs = [subs for subs in top_subs if subs[5] < 0.45]
if not top_subs:
    print("❌ No Relevant submissions found")
else:
    print(f"✅ Found {len(top_subs)} relevant submissions")

# Printing submissions
print(json.dumps(top_subs, indent=5))

Retrieveing top matching submissions....
✅ Found 15 relevant submissions
[
     [
          "a2dmje",
          "953de4be-d2f0-4ca2-9a89-252ce61ad883",
          "I stumbled upon an interesting post on Instagram about ideal body shapes throughout history - Link Included!",
          "[Hopefully you can all see it! ](https://www.instagram.com/p/BqsSXu9htZG/?utm_source=ig_share_sheet&igshid=rtfm5hl48fos)\n\nIt really illustrates the way society shapes our perception of what we should look like based on what the world deems as \u201cin\u201d. It makes me sad, but also makes me feel a little more self aware. \nI have definitely fallen victim to wanting to change myself to fit a mold even if it is unrealistic for me and my genetics. \n\nThis past 6 months I\u2019ve been going back and forth with different goals, struggling to find balance, but trying my hardest to accept myself in the process of finding this balance. More than ever, I want to accept myself NOW as well as choose to change my

In [5]:
# Collect related comments
context_blocks = []
sources = []  # track all permalinks for final output

top_submission_ids = [row[0] for row in top_subs]
print(top_submission_ids)


# Pulling in the most Relevant comments from only the top posts that we get
# There is also a small scoring boost for the high upvoted comments, at the time of querying
# Relevance first tho, only then popularity
query = """
WITH ranked_comments AS (
    SELECT 
        c.body,
        c.score,
        s.title,
        c.submission_id,
        s.permalink,
        c.embedding <=> %s::vector AS distance
    FROM comments c
    JOIN submissions s ON c.submission_id = s.submission_id
    WHERE c.submission_id = ANY(%s)
)
SELECT 
    body,
    score,
    title,
    submission_id,
    permalink,
    distance
FROM ranked_comments
ORDER BY 
    distance + (1.0 / (score + 10)) ASC 
LIMIT 20;
"""

# Note the order of parameters is now --> embedding, submission_ids list
cursor.execute(query, (json.dumps(q_emb), top_submission_ids))
relevant_comments = cursor.fetchall()

print(f"\n✅ Retrieved {len(relevant_comments)} relevant comments\n")

# for i, (body, score, title, permalink, distance) in enumerate(relevant_comments, 1):
#     print(f"{i:2d}. ↑{score}  (dist: {distance:.4f}) | {title}")
#     print(f"     {body.strip()[:900]}{'...' if len(body) > 900 else ''}")
#     print(f"     https://reddit.com{permalink}\n")
# print("="*120)

# Context map contains : submission_id -> {submission_info, comments}
context_map = {}

# building submission details
submission_details = {}
for sub in top_subs:
    submission_id = sub[0]
    title = sub[2]
    selftext = sub[3]
    permalink = sub[4]
    
    submission_details[submission_id] = {
        'title': title,
        'selftext': selftext,
        'permalink': permalink
    }


for comment in relevant_comments:
    body = comment[0]
    score = comment[1]
    title = comment[2]
    submission_id = comment[3]
    permalink = comment[4]
    distance = comment[5]
    
    if submission_id not in context_map:
        # Get selftext from submission_details
        sub_info = submission_details.get(submission_id, {})
        
        context_map[submission_id] = {
            'title': title,
            'permalink': permalink,
            'selftext': sub_info.get('selftext', ''),
            'comments': []
        }
    
    context_map[submission_id]['comments'].append({
        'body': body,
        'score': score,
        'distance': distance
    })

print(f"✅ Built context map with {len(context_map)} submissions\n")
print(json.dumps(context_map, indent=5))

['9hjn8y', 'a2dmje', '8hht7o', '93kcov', 'z9q8hm', '85tpgh', 'dl265l', '1kc67ha', '995lst', '1fitbc5', '631os1', 'juou3v', 'a8raia', '7nrqkz', '9t0gqz']

✅ Retrieved 20 relevant comments

✅ Built context map with 9 submissions

{
     "9hjn8y": {
          "title": "Everything I've Learned About Obesity is Right (and How I Used That To Lose 150lbs)",
          "permalink": "https://www.reddit.com/r/loseit/comments/9hjn8y/everything_ive_learned_about_obesity_is_right_and/",
          "selftext": "##How It Started\n\nThe date is June 30th, 2017. My name is Amanda and I weigh 331lbs. I just finished up an appointment at the gynecologist to get a mirena put in. My gynecologist, a sweet asian-american in her early 30s with a healthy body and kind face, tells me that the procedure is finished and I am good to go, but... \n\n\"Your blood pressure is awfully high, I am worried about you,\" she adds quickly. \"It was high last time, too. You should really see a doctor, there is one here, I can 

# Looking at more fetched comment details


In [6]:
# Prepare for generation, 
context_blocks = []
sources = []

for submission_id, data in context_map.items():
    title = data['title']
    permalink = data['permalink']
    selftext = data['selftext']
    comments = data['comments']
    
    # Format comments
    joined_comments = "\n".join([
        f"- (↑{c['score']}) {c['body'][:600]}..." 
        for c in comments
    ])
    
    # Build block
    block = f"""
[POST] "{title}"
URL: {permalink}

{selftext[:1200] if selftext else "(No text body)"}

Top Comments:
{joined_comments}
"""
    context_blocks.append(block)
    sources.append(f"- {title}: {permalink}")

# Final context text to pass to LLM
context_text = "\n\n---\n\n".join(context_blocks)

print("CONTEXT FOR LLM:")
print(context_text)
print("\n" )
print("SOURCES:")
print("\n".join(sources))

CONTEXT FOR LLM:

[POST] "Everything I've Learned About Obesity is Right (and How I Used That To Lose 150lbs)"
URL: https://www.reddit.com/r/loseit/comments/9hjn8y/everything_ive_learned_about_obesity_is_right_and/

##How It Started

The date is June 30th, 2017. My name is Amanda and I weigh 331lbs. I just finished up an appointment at the gynecologist to get a mirena put in. My gynecologist, a sweet asian-american in her early 30s with a healthy body and kind face, tells me that the procedure is finished and I am good to go, but... 

"Your blood pressure is awfully high, I am worried about you," she adds quickly. "It was high last time, too. You should really see a doctor, there is one here, I can give you a recommendation. He's very business-oriented and scientific, you'll like him." 

Hesitantly, I agree. This is not the first time someone has spoken to me about my blood pressure, but it will be the first time I have visited a doctor in almost 8 years. 

Thyroid issues, blood pressu

In [7]:
print("Generating output...")
prompt = f"""
    You are an empathetic assistant that uses examples from real Reddit users' discussions
    to guide someone based on their current life goal.

    User Query:
    {user_input}

    Here are similar cases from Reddit:
    {context_text}

    Using the experiences and insights quoted above, write an actionable plan
    to help the user.
    At the END of your response, include a "Sources" section listing all the Reddit discussions you referenced.
    Format each source as: "- [Post Title]: [full permalink URL]"
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful, grounded life-strategy coach."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

print("\n--- LLM RESPONSE ---\n")
print(response.choices[0].message.content)

Generating output...

--- LLM RESPONSE ---

Absolutely! Losing weight can be a challenging journey, but with the right mindset and actionable steps, it’s definitely achievable. Here’s an actionable plan inspired by real experiences shared by Reddit users.

### Actionable Weight Loss Plan

1. **Set Clear, Realistic Goals**: 
   - Define what weight loss means to you. Rather than just focusing on numbers, think about how you want to feel and the lifestyle changes you wish to make. For example, Amanda (from her weight loss journey) realized that her health issues were a wake-up call and led her to seek medical advice and change her lifestyle.

2. **Focus on Nutrition**:
   - Transition into healthier eating habits rather than starting a strict diet. For instance, one Redditor shared how they stopped consuming soda except for once a week and focused on shopping the perimeter of the grocery store, which typically has fresh produce and whole foods. Making small, manageable changes can lead t